In [318]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [319]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import collections
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pysam
import scipy as sp
import seaborn
import sharedmem
import pygraphviz

In [320]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from data_io import is_fwd_id, get_fwd_id, get_sibling_id
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges
from truth import get_overlaps
from evaluate import NearestNeighborsConfig, mp_compute_nearest_neighbors
from plots import plot_read_graph, mp_plot_read_graphs, get_graphviz_layout, get_umap_layout

In [321]:
from memory_profiler import profile
%load_ext memory_profiler


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [322]:
%memit x = 10+5

peak memory: 84776.26 MiB, increment: 0.00 MiB


In [ ]:
MAX_SAMPLE_SIZE = int(1e9)
COVERAGE_DEPTH = 20
max_n_neighbors = 20
with gzip.open(fasta_gz_file, "rt") as handle:
    flag = 0
    seq_num = 0
    aligned_num = 0
    for record in SeqIO.parse(handle, "fasta"):
        seq_num += 1
        if record.id in max_values.keys():
            aligned_num+=1
            columns = max_values[record.id]
            if int(columns[9])/int(columns[1]) > 0.5:
                flag += 1
    print(flag/seq_num)
    print(aligned_num/seq_num)

In [ ]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k//kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/CHM13/HLA/pbsim_ONT_93_30k/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

In [ ]:
feature_matrix.shape

(7448, 1265400)

In [ ]:
##original version 
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index) 
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 40, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]



299897765
2378963497
44873935
2670060064
25935175
920060685
1586326711
1550303102
278283145
1103905279
2497529993
993549511
3538826777
827898949
1744767203
1454149367
347405931
293704892
739522939
2787976300
923371613
3028338653
1487664578
1269177166
1108403661
2333147459
3468047501
3767013626
3224645638
216444033
873431322
2428674822
205263707
3132866966
3313916789
964022396
3691436413
768347323
2334284253
3571135595


In [ ]:
 np.count_nonzero(concat_simhash == 0)/concat_simhash.size 

0.505719236707841

In [ ]:
nbr_indices

array([[6175, 3019, 2411, ..., 2473, 5045, 1522],
       [6174, 2410, 3018, ..., 4218,  254, 1379],
       [2989, 2397, 4921, ..., 5669, 5670, 4486],
       ...,
       [ 426, 5654, 7069, ..., 5105, 1101, 4029],
       [ 549, 1398, 3729, ...,  853, 6298,  402],
       [1399, 3728, 5923, ..., 4456, 5067, 6947]])

In [ ]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [ ]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graphs[k] = graph

In [ ]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(7448, 150966, 14394, 2)

In [ ]:
df_rows = []

for k in k_values:
    graph = graphs[k]
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, 
                 **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
#new hash 
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.005842,0.000537,0.000576,0.000556,7446,0.999731,1
1,SimHash,3,0.006357,0.000492,0.000941,0.000764,7445,0.999597,1
2,SimHash,4,0.006044,0.000504,0.001192,0.001042,7444,0.999463,1
3,SimHash,5,0.005964,0.000510,0.001471,0.001320,7443,0.999329,1
4,SimHash,6,0.005619,0.000515,0.001663,0.001598,7442,0.999194,1
5,SimHash,7,0.005508,0.000518,0.001901,0.001876,7441,0.999060,1
6,SimHash,8,0.005307,0.000521,0.002093,0.002154,7440,0.998926,1
7,SimHash,9,0.005658,0.000522,0.002510,0.002432,7439,0.998792,1
8,SimHash,10,0.005482,0.000524,0.002703,0.002709,7438,0.998657,1
9,SimHash,11,0.005583,0.000525,0.003027,0.002987,7437,0.998523,1


In [ ]:
##20
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.990493,0.374306,0.077984,0.309087,74,0.009936,142
1,SimHash,3,0.988986,0.335633,0.113012,0.402251,57,0.007653,462
2,SimHash,4,0.987583,0.307521,0.146993,0.480061,46,0.006176,1904
3,SimHash,5,0.986380,0.286463,0.179411,0.546478,42,0.005639,3664
4,SimHash,6,0.985000,0.266570,0.210961,0.598791,37,0.004968,3666
5,SimHash,7,0.983400,0.251502,0.241730,0.648395,35,0.004699,3666
6,SimHash,8,0.981522,0.238416,0.271637,0.692024,33,0.004431,3666
7,SimHash,9,0.979438,0.226441,0.300379,0.728359,32,0.004296,3713
8,SimHash,10,0.977118,0.215884,0.328683,0.761637,32,0.004296,3714
9,SimHash,11,0.974350,0.206200,0.356040,0.790260,31,0.004162,3714


In [ ]:
##40
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.996473,0.392710,0.078594,0.324858,31,0.004162,144
1,SimHash,3,0.996362,0.347693,0.114291,0.418299,16,0.002148,517
2,SimHash,4,0.995634,0.319150,0.148020,0.497638,13,0.001745,1684
3,SimHash,5,0.994618,0.295200,0.181180,0.563985,12,0.001611,1684
4,SimHash,6,0.994103,0.275716,0.213300,0.620467,7,0.000940,3352
5,SimHash,7,0.993436,0.260578,0.244631,0.672989,6,0.000806,3721
6,SimHash,8,0.992592,0.246439,0.275122,0.716410,6,0.000806,3722
7,SimHash,9,0.992042,0.234327,0.304718,0.754898,5,0.000671,3722
8,SimHash,10,0.991456,0.223456,0.333578,0.788523,5,0.000671,3722
9,SimHash,11,0.990338,0.213041,0.361870,0.816451,4,0.000537,3722


In [ ]:
def universal_hash(x, seed):
    prime1 = 2654435761  # A large prime number
    prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
    hash_value = (x * prime1) ^ (seed * prime2)
    return hash_value % (2**32)

# Example usage
input_number = 123456
seed = 42
hashed_number = universal_hash(input_number, seed)
print(f"Input: {input_number}, Seed: {seed}, Hashed: {hashed_number}")

'1111111111111111111111111111111110010100000101101010110010010011'

In [ ]:
##version1 fill the hash table
epeat = 20

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

all_read_simhash = []
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)
kmer_num = feature_matrix.shape[1]
hash_table = np.empty((kmer_num,repeat),dtype=object)  
for flag,seed in enumerate(hash_seeds):
    print(flag)
    kmer_hash_indice = {} 
    for kmer_index in range(kmer_num):
        hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
    conc_hash = np.concatenate(one_read_hash)
    simhash = np.where(conc_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
reads_simhash_array

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 1, 1]])

In [ ]:
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

In [ ]:
hash_seeds = np.array(range(repeat))
kmer_num = feature_matrix.shape[1]
kmer_index = np.array(range(kmer_num))

table = []
for seed in hash_seeds:
    prime1 = 2654435761  # A large prime number
    prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
    hash_value = (kmer_index * prime1) ^ (seed * prime2)
    hash_t = hash_value % (2**32)
    binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
    one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
    table.append(one_repear_table)


In [ ]:
pip install sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 784.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 659.6 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sympy import randprime

def generate_large_prime(bits=33):
    # Lower bound is 2^(bits-1), upper bound is 2^bits - 1
    lower_bound = 2**(bits-1)
    upper_bound = 2**bits - 1
    
    # Generate a random prime number in the given range
    prime = randprime(lower_bound, upper_bound)
    
    return prime

# Example usage
prime = generate_large_prime(33)  # Generate a prime number larger than 2^32
print(f'Generated prime: {prime}')

import random

class UniversalHash:
    def __init__(self, size, prime, seed=None):
        self.size = size  # Size of the hash table
        self.prime = prime  # A prime number larger than the maximum expected value
        self.seed = seed  # Seed for reproducibility
        
        if seed is not None:
            random.seed(seed)  # Set the seed for the random number generator
        
        self.a = random.randint(1, prime - 1)  # a should be non-zero
        self.b = random.randint(0, prime - 1)  # b can be any value in the range [0, p-1]
    
    def hash(self, x):
        return ((self.a * x + self.b) % self.prime) % self.size

# Example usage
size = 10  # Size of the hash table
seed = 42  # Seed for reproducibility

uh = UniversalHash(size, prime, seed)

# Hash some values
for x in [1, 2, 3, 100, 200, 300,322,433]:
    print(f'hash({x}) = {uh.hash(x)}')

Generated prime: 4778274367
hash(1) = 2
hash(2) = 9
hash(3) = 3
hash(100) = 9
hash(200) = 0
hash(300) = 4
hash(322) = 8
hash(433) = 4


In [ ]:
prime

6985678901

In [ ]:
##version2 numpy calculate hash values
repeat = 20
rng = np.random.default_rng(seed)  
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  

kmer_num = feature_matrix.shape[1]
kmer_index = np.array(range(kmer_num))

table = []
for seed in hash_seeds:
    prime1 = 2654435761  # A large prime number
    prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
    hash_value = (kmer_index * prime1) ^ (seed * prime2)
    hash_t = hash_value % (2**32)
    binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
    one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
    table.append(one_repear_table.astype(int))

hash_table1 = np.hstack(table)
hash_table = np.where(hash_table1 ==0,-1,1)

all_read_simhash = []

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
    simhash = np.where(one_read_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)

TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:

nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
graph_stats

{'precision': 0.13479035130322162,
 'nr_precision': 0.02906481031784577,
 'recall': 0.0827272365963197,
 'nr_recall': 0.18709184382381547,
 'singleton_count': 2545,
 'singleton_fraction': 0.34170247046186897,
 'N50': 65}

In [ ]:
one_Read = []
sim = []
for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
    one_Read.append(one_read_hash)
    simhash = np.where(one_read_hash > 0, 1, 0)
    sim.append(simhash)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)